<a href="https://colab.research.google.com/github/adolfoguimaraes/inteligenciaartificial-oficina/blob/main/models/notebooks/OpenAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import openai
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read("../../config.ini")
openai.api_key = config['OPENAI']['SECRETKEY']

In [ ]:
def deleteFile(id):

    return_ = openai.File.delete(id)

    if (return_['deleted']):
        print("Arquivo %s deletado." % id)
    else:
        print("Arquivo %s não deletado." % id)



In [ ]:
def createFile(input):


    data_file = openai.File.create(
        file=open(input),
        purpose="answers"
    )

    print("Arquivo %s criado." % data_file['id'])

    return data_file

O método makeAnswer encapsula a função que envia os dados para a OpenAI e retorna a resposta. Para isso, precisamos informar qual o `input_file` (arquivo com os pares perguntas e respostas) e a `question` (a pergunta que será enviada).

In [ ]:
def makeAnswer(input_file, question, example_context, examples):

    try:
        return_ = openai.Answer.create(
            search_model="davinci",
            model="davinci",
            question=question,
            file=input_file,
            examples_context=example_context,
            examples=examples,
            max_tokens=50,
            stop=["\n", "<|endoftext|>"]
        )
    except Exception as e:
        print("Não foi possível responder a sua pergunta.")
        print(e)
        return_ = []

    
    return return_['answers'][0]

    

## Exemplo simples de conversação.

Para esse exemplo, vamos utilizar o arquivo `data.jsonl` que está na pasta `datasets/conversation`. Ele tem alguns pares de perguntas e respostas sobre uma pessoa. A idéia é verificar se com esse conjunto de textos, a máquina consegue responder as perguntas. 

In [ ]:
# Cria o arquivo para conversação.
dataFile = createFile("../datasets/conversation/data.jsonl")

Arquivo file-WfifZ6GkEVEitouNTddTfXUy criado.


Vamos fazer algumas perguntas: 

* Qual a profissão de João e onde ele trabalha?
* Com quem João é casado?
* O que a esposa de João faz?
* João pretende ter filhos?
* O que João gosta de assistir? 
* João gosta de assistir Friends. E sua esposa? 
* João ama quem? 
* Qual o nome do filho de João?


In [ ]:
example_context_1 = "João gosta de assistir a série Friends."
examples_1 = [["O que João assiste na TV?","A série Frieds."]]

In [ ]:
makeAnswer(dataFile['id'],'Qual a profissão de João e onde ele trabalha?', example_context_1, examples_1)

'Ele é professor universitário.'

# Exemplo com uma base maior

Vamos trabalhar com um contexto um pouco diferente. A proposta é criar um arquivo com uma série de perguntas e respostas sobre um determinado filme e ver como o modelo responde a perguntas relacionadas. Para montar o arquivo `starwars.jsonl`, utilizei algumas das perguntas e respostas disponíveis nestes links: 

* https://parade.com/1161189/alexandra-hurtado/star-wars-trivia/.
* https://bigquizthing.com/blog-post/star-wars-trivia-questions-for-fun-game-night/

In [ ]:
# Limpando a lista de arquivos e criando o arquivo com os dados da conversação.
dataFile2 = createFile("../datasets/conversation/starwars.jsonl")

Arquivo file-ruakplgi1ad3SkF2osaaD6Sg criado.


In [ ]:
example_context_2 = "Star Wars is a famous movie's serie. There are a lot of fans around the world"
examples_2 = [["What is Star Wars?","A famous movie's serie."]]

makeAnswer(dataFile2['id'],'What you know about Anakin Skywalker?', example_context_2, examples_2)

'He is a famous Jedi.'